# Compras Governamentais - MIRA

Acessando os dados das nfe do portal da transparência da CGU.

In [1]:
import duckdb, pandas as pd

from IPython.display import display
import numpy as np
from decimal import Decimal

pd.set_option("display.float_format", lambda x: f"{x:_.2f}") 
def underscore_fmt(x):
    if pd.isna(x): return ""
    if isinstance(x, (int, np.integer)):   return f"{x:_d}"
    if isinstance(x, (float, np.floating)): return f"{x:_.0f}"
    if isinstance(x, Decimal):             return f"{x:_.0f}"
    return x

In [2]:

db = duckdb.connect(database='../data/outputs/mgi/dw.db')

In [20]:

for e in ("spatial","httpfs","json","excel"): db.install_extension(e); db.load_extension(e)
for e in ("zipfs",): db.install_extension(e, repository="community"); db.load_extension(e)

In [3]:
db.execute("""ATTACH '../data/outputs/cgu/nfe.db' as nfe""")
db.execute("""select * from nfe.cgu.notas limit 2""").df()

,chave_acesso,modelo,serie,numero,natureza_operacao,timestamp_emissao,data_emissao,mes_emissao,ano_emissao,ultimo_evento,...,orgao_destinatario,destinatario,nome_destinatario,uf_destinatario,indicador_ie_destinatario,destino_operacao,indicador_consumidor_final,indicador_presenca_comprador,valor_nota_fiscal,duplicada
0,42220107432517001847550030005499571000224173,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,3,549957,Outra saida merc./prest.serv. nao especif.,2022-01-01 00:02:22,2022-01-01,1,2022,Autorização de Uso,...,Sem informação,00394460043975,MINISTERIO DA ECONOMIA,DF,NÃO CONTRIBUINTE,2 - OPERAÇÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,0 - NÃO SE APLICA,62.14,False
1,42220107432517001847550030005499821000227345,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,3,549982,Outra saida merc./prest.serv. nao especif.,2022-01-01 00:03:59,2022-01-01,1,2022,Autorização de Uso,...,"Instituto Federal de Educação, Ciência e Tecno...",10779511000298,"INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNO...",RJ,CONTRIBUINTE ISENTO,2 - OPERAÇÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,0 - NÃO SE APLICA,224.42,False


In [4]:
canceladas = db.execute("""SELECT chave_acesso from nfe.cgu.eventos where evento = 'Cancelamento da NF-e'""").df()
canceladas.head(2)

,chave_acesso
0,11220103552842000144550000007133711021401139
1,11220103983300000550550010000486931172800086


Note que há notas que não são de compras, mas apenas de transferência dentro do mesmo órgão.

In [44]:
db.execute("select chave_acesso, natureza_operacao, emitente, nome_emitente, destinatario, nome_destinatario from nfe.cgu.notas where emitente = destinatario").df()

,chave_acesso,natureza_operacao,emitente,nome_emitente,destinatario,nome_destinatario
0,31220110648539000296550010000039731554575770,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
1,31220110648539000296550010000039741177514000,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
2,15220134621748000638550010000060251000398021,TRANSFERÊNCIA PARA COMERCIALIZACAO,34621748000638,EDITORA DA UFPA,34621748000638,UNIVERSIDADE FEDERAL DO PARA - EDITORA UFPA
3,53220100038174000496550010000120881001461526,"DOACAO, BONIFICACAO OU BRINDE",00038174000496,FUNDACAO UNIVERSIDADE DE BRASILIA,00038174000496,FUNDACAO UNIVERSIDADE DE BRASILIA
4,31220110648539000296550010000039751325780273,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
...,...,...,...,...,...,...
4569,31241210648539000296550010000061021136455891,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
4570,31241217217985004282550010000414321000228601,INDUST FEITA POR TERCEIROS,17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS,17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS
4571,31241217217985004282550010000414331000228617,"ENTRADA DE BONIFICACAO, DOACAO OU BRINDE",17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS,17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS
4572,31241210648539000296550010000061041276387656,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)


In [46]:
db.execute("select chave_acesso, natureza_operacao, emitente, nome_emitente, destinatario, nome_destinatario from nfe.cgu.notas where left(emitente,8) = left(destinatario,8)").df()

,chave_acesso,natureza_operacao,emitente,nome_emitente,destinatario,nome_destinatario
0,31220110648539000296550010000039731554575770,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
1,43220187958674000181558900383383291813921498,Saídas Diversas,00662270000915,INST NAC METROL QUALIDADE E TECNOLOGIA INMETRO,00662270000320,"INSTITUTO NACIONAL DE METROLOGIA, QUALIDADE E ..."
2,33220142498675000152558900041721101288054913,Outras saídas de mercadorias ou prestações de ...,00662270000320,'INSTITUTO NACIONAL DE METROLOGIA,00662270000915,INST NACIONAL DE METROLOGIA QUALIDADE INMETRO
3,31220110648539000296550010000039741177514000,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
4,31220110648539000105550010000000531060950904,DEVOLUÇÃO BEM ATIVO IMOBILIZADO,10648539000105,INSTITUTO FEDERAL DE EDUCACAO CIENCIA E TECNOL...,10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
...,...,...,...,...,...,...
7535,31241217217985004282550010000414321000228601,INDUST FEITA POR TERCEIROS,17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS,17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS
7536,31241217217985004282550010000414331000228617,"ENTRADA DE BONIFICACAO, DOACAO OU BRINDE",17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS,17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS
7537,53241200038174001468550010000005721001690177,TRANSFERENCIA PRODUTO SAIDA,00038174001468,FUNDACAO UNIVERSIDADE DE BRASILIA,00038174001972,FUNDACAO UNIVERSIDADE DE BRASILIA - LIVRARIA
7538,31241210648539000296550010000061041276387656,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)


## Filtrando a base CGU

In [47]:
compras_sql = """
           SELECT * FROM nfe.cgu.notas where ultimo_evento not in (
                'Cancelamento da NF-e', 
                'Manifestação do destinatário - Operação não realizada', 
                'Manifestação do destinatário - Desconhecimento da operação')
            AND chave_acesso NOT IN (SELECT chave_acesso from nfe.cgu.eventos where evento = 'Cancelamento da NF-e')
            AND chave_acesso NOT IN (SELECT chave_acesso from nfe.cgu.notas where left(emitente,8) = left(destinatario,8))
"""
db.execute("CREATE SCHEMA IF NOT EXISTS mgi")
db.execute(f"CREATE OR REPLACE TABLE mgi.notas AS {compras_sql}").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,4983258


In [48]:
db.query("SELECT ano_emissao, COUNT(*) as notas, SUM(valor_nota_fiscal) as valor_total FROM mgi.notas group by all").df()

,ano_emissao,notas,valor_total
0,2022,1639023,75_617_577_515.53
1,2023,1715084,71_704_876_669.73
2,2024,1629151,67_102_506_351.88


Não exatamente o que aparece no portal da transparência, mas é o que temos no momento.

In [7]:
itens_sql = """
    SELECT * from nfe.cgu.itens where chave_acesso IN (SELECT chave_acesso FROM mgi.notas)
"""
display(db.execute(f"CREATE OR REPLACE TABLE mgi.itens AS {itens_sql}").df().style.format(underscore_fmt))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,18_092_434


In [8]:
db.query("SELECT ano_emissao, COUNT(*) as itens, SUM(valor_item) as valor_total FROM mgi.itens group by all").df()

,ano_emissao,itens,valor_total
0,2022,5519989,80_392_651_853.44
1,2023,6046484,81_776_271_025.25
2,2024,6525961,80_337_907_202.54


In [9]:
diferentes = db.query("""
SELECT
  i.chave_acesso,
  SUM(i.valor_item)         AS valor_itens,
  MAX(n.valor_nota_fiscal)  AS valor_nota,
  CASE 
    WHEN SUM(i.valor_item) = 0 THEN NULL
    ELSE 100.0 * (SUM(i.valor_item) - MAX(n.valor_nota_fiscal)) / SUM(i.valor_item)
  END AS diff_pct
FROM mgi.itens i
JOIN mgi.notas n USING (chave_acesso)
GROUP BY i.chave_acesso
HAVING ABS(SUM(i.valor_item) - MAX(n.valor_nota_fiscal)) > 0.01
ORDER BY i.chave_acesso
""").df()
diferentes

,chave_acesso,valor_itens,valor_nota,diff_pct
0,11220101055747000100550010000270531548085752,87.10,79.08,9.21
1,11220102436230000123550010000033411264818401,104.00,95.12,8.54
2,11220102436230000123550010000033421531465693,1_001.00,965.47,3.55
3,11220102436230000123550010000033461668287790,1_042.00,1_031.58,1.00
4,11220102436230000123550010000033581927160082,4_170.00,4_167.91,0.05
...,...,...,...,...
728693,53241272591043000108550010000041971009593958,1_750.00,1_563.45,10.66
728694,53241272591043000108550010000041991009610660,750.00,670.05,10.66
728695,53241275315333006140550010018329491758422969,321.99,321.94,0.02
728696,53241275315333006140550010018355301758499198,309.76,309.86,-0.03


In [10]:
diferentes.describe()

,valor_itens,valor_nota,diff_pct
count,728_698.00,728_698.00,687_471.00
mean,104_857.40,71_954.98,-419.13
std,1_930_192.65,1_260_564.02,323_091.30
min,0.00,0.00,-267_554_500.00
25%,385.02,376.95,-1.75
50%,1_687.24,1_588.61,0.00
75%,9_241.80,8_790.36,8.64
max,338_990_778.36,215_831_701.43,100.00


In [11]:
# Top 20 most negative diff_pct rows (excluding NULLs)
db.query("""
WITH agreg AS (
  SELECT
    i.chave_acesso,
    SUM(i.valor_item)        AS valor_itens,
    MAX(n.valor_nota_fiscal) AS valor_nota
  FROM mgi.itens i
  JOIN mgi.notas n USING (chave_acesso)
  GROUP BY 1
),
calc AS (
  SELECT
    chave_acesso,
    valor_itens,
    valor_nota,
    CASE WHEN valor_itens = 0 THEN NULL
         ELSE 100.0 * (valor_itens - valor_nota) / NULLIF(valor_itens, 0)
    END AS diff_pct
  FROM agreg
)
SELECT *
FROM calc
WHERE diff_pct IS NOT NULL
ORDER BY diff_pct ASC
LIMIT 20
""").df()

,chave_acesso,valor_itens,valor_nota,diff_pct
0,35241001382559000196550010000568791453969473,0.01,26_755.46,-267_554_500.00
1,33220267405936000173550030007312531090919160,0.01,1_150.86,-11_508_500.00
2,33231242498675000152558900082727141232845850,0.08,4_500.08,-5_625_000.00
3,33241042498675000152558900095996841388073319,0.03,1_074.03,-3_580_000.00
4,33241042498675000152558900096004241456950049,0.06,537.06,-895_000.00
5,33241042498675000152558900096000581631737660,0.09,537.09,-596_666.67
6,43231091967067000155550000004604271397197016,0.01,35.52,-355_100.00
7,35220859231530000193550010000963991554476272,0.06,115.71,-192_750.00
8,53241128843315000110550010000022981105953029,0.70,1_100.70,-157_142.86
9,33231242498675000152558900082646331199348194,0.80,1_000.80,-125_000.00


In [12]:
db.execute("select chave_acesso, timestamp_emissao, descricao_produto, ncm, valor_item FROM mgi.itens where chave_acesso = '35241001382559000196550010000568791453969473'").df()

,chave_acesso,timestamp_emissao,descricao_produto,ncm,valor_item
0,35241001382559000196550010000568791453969473,2024-10-25 13:10:00,ANALISADOR DE TEOR DE FAME EM COMBUSTIVEL DE A...,90278999,0.01


In [13]:
db.execute("select chave_acesso, timestamp_emissao, valor_nota_fiscal FROM mgi.notas where chave_acesso = '35241001382559000196550010000568791453969473'").df()

,chave_acesso,timestamp_emissao,valor_nota_fiscal
0,35241001382559000196550010000568791453969473,2024-10-25 13:10:00,26_755.46


Em geral, o valor_nota é menor que o valor_itens, mas há excecões 

In [14]:
db.query("""
SELECT
  i.chave_acesso,
  SUM(i.valor_item)         AS valor_itens,
  MAX(n.valor_nota_fiscal)  AS valor_nota
FROM mgi.itens i
JOIN mgi.notas n USING (chave_acesso)
where i.chave_acesso = '33221200776574001390550050329327331446711857'
GROUP BY i.chave_acesso
HAVING ABS(SUM(i.valor_item) - MAX(n.valor_nota_fiscal)) > 0.01
ORDER BY i.chave_acesso
""").df()

,chave_acesso,valor_itens,valor_nota
0,33221200776574001390550050329327331446711857,3_467.49,3_427.73


O valor do item não bate com a nota porque o item não está considerando valores de frete e desconto como visto abaixo:


![](./item_vs_nota.png)

In [15]:
db.execute("""
SELECT
  i.chave_acesso,
  SUM(i.valor_item)         AS valor_itens,
  MAX(n.valor_nota_fiscal)  AS valor_nota,
  100.0 * (SUM(i.valor_item) - MAX(n.valor_nota_fiscal)) / NULLIF(SUM(i.valor_item), 0) AS diff_pct
FROM mgi.itens i
JOIN mgi.notas n USING (chave_acesso)
GROUP BY i.chave_acesso
HAVING ABS( (SUM(i.valor_item) - MAX(n.valor_nota_fiscal)) / NULLIF(SUM(i.valor_item), 0) ) > 0.10
ORDER BY diff_pct DESC
           """).df()

,chave_acesso,valor_itens,valor_nota,diff_pct
0,35220628806187000134550010000018981861429549,604.80,0.00,100.00
1,41230713805697000110550010000317511328870283,136.70,0.00,100.00
2,33230700323090000313550020000025031283928813,17_084.10,0.00,100.00
3,41230713805697000110550010000314001300838258,131.70,0.00,100.00
4,25230816713807000101550010000031911947908580,149.45,0.00,100.00
...,...,...,...,...
196109,33241042498675000152558900096004241456950049,0.06,537.06,-895_000.00
196110,33241042498675000152558900095996841388073319,0.03,1_074.03,-3_580_000.00
196111,33231242498675000152558900082727141232845850,0.08,4_500.08,-5_625_000.00
196112,33220267405936000173550030007312531090919160,0.01,1_150.86,-11_508_500.00


In [16]:
196118/4990754 * 100.0

3.929626665630083

4% das notas tem valor_itens mais de 10% diferente do valor_nota, 2% mais de 20% diferente e 1% mais de 30% diferente. Vamos usar como base mgi.itens mas não estamos filtrando onde valor_item é diferente de valor_nota_fiscal.


### NCM

In [18]:
db.execute("""
    CREATE OR REPLACE TABLE mgi.ncm AS
    SELECT
        unnest."Codigo" AS ncm,
        unnest."Descricao" AS ncm_nome,
        strptime(unnest."Data_Inicio", '%d/%m/%Y')::date AS data_inicio
    FROM
        read_json('https://portalunico.siscomex.gov.br/classif/api/publico/nomenclatura/download/json'),
        UNNEST(Nomenclaturas) AS unnest
        """).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,15141


In [23]:
db.query("FROM mgi.ncm").df()

,ncm,ncm_nome,data_inicio
0,01,Animais vivos.,2022-04-01
1,01.01,"Cavalos, asininos e muares, vivos.",2022-04-01
2,0101.2,- Cavalos:,2022-04-01
3,0101.21.00,-- Reprodutores de raça pura,2022-04-01
4,0101.29.00,-- Outros,2022-04-01
...,...,...,...
15136,9705.31.00,-- Com mais de 100 anos,2022-04-01
15137,9705.39.00,-- Outras,2022-04-01
15138,97.06,Antiguidades com mais de 100 anos.,2022-04-01
15139,9706.10.00,- Com mais de 250 anos,2022-04-01


de_ncm_para_scn128: [Tradutor Nomenclatura Comum do Mercosul (NCM) - Comércio Exterior/Contas Nacionais ](https://ftp.ibge.gov.br/Contas_Nacionais/Sistema_de_Contas_Nacionais/Tradutores/TradutorNCM.xls)
Ver: https://www.ibge.gov.br/estatisticas/economicas/servicos/9052-sistema-de-contas-nacionais-brasil.html?=&t=notas-tecnicas

In [24]:
db.execute("""
    CREATE OR REPLACE TABLE mgi.de_ncm_para_scn AS
    SELECT
        *
    FROM
        read_parquet('../data/inputs/mgi/de_ncm_para_scn128.parquet')
        """).df()

,Count
0,11875


In [26]:
db.execute("SELECT * FROM mgi.de_ncm_para_scn").df().head(3)

,NCM_COD,NCM_NOME,SCN128_COD,SCN128_DESC
0,01011010,Cavalos reprodutores de raça pura,01921,"Bovinos e outros animais vivos, prods. animal,..."
1,01011090,"Animais da espécie asinina/muar, reprodutores,...",01921,"Bovinos e outros animais vivos, prods. animal,..."
2,01011100,"Cavalos reprodutores, de raça pura",01921,"Bovinos e outros animais vivos, prods. animal,..."


Nota: Falta explicar como a tabela abaixo foi criada.

In [27]:
db.execute("""
    CREATE OR REPLACE TABLE mgi.scn AS
    SELECT
        *
    FROM
        read_parquet('../data/inputs/mgi/scn.parquet')       
        """)
db.execute("""
           SELECT *
           FROM mgi.scn
""").df()

,SCN128_COD,SCN128_NOME,SCN67_COD,SCN67_NOME
0,01911,"Arroz, trigo e outros cereais",0191,"Agricultura, inclusive o apoio à agricultura e..."
1,01912,Milho em grão,0191,"Agricultura, inclusive o apoio à agricultura e..."
2,01913,"Algodão herbáceo, outras fibras da lav. tempor...",0191,"Agricultura, inclusive o apoio à agricultura e..."
3,01914,Cana-de-açúcar,0191,"Agricultura, inclusive o apoio à agricultura e..."
4,01915,Soja em grão,0191,"Agricultura, inclusive o apoio à agricultura e..."
...,...,...,...,...
123,90801,"Serviços de artes, cultura, esporte e recreação",9080,"Atividades artísticas, criativas e de espetáculos"
124,94801,"Organizações patronais, sindicais e outros ser...",9480,Organizações associativas e outros serviços pe...
125,94802,"Manutenção de computadores, telefones e objeto...",9480,Organizações associativas e outros serviços pe...
126,94803,Serviços pessoais,9480,Organizações associativas e outros serviços pe...


### CFOP

A tabela CFOP está em: [https://www.nfe.fazenda.gov.br > Documentos > Diversos](https://www.nfe.fazenda.gov.br/portal/listaConteudo.aspx?tipoConteudo=/NJarYc9nus=)

`https://www.nfe.fazenda.gov.br/portal/exibirArquivo.aspx?conteudo=UGAhInpwpPw=`

Mas a fonte mais completa de informações sobre CFOP está em: https://www.confaz.fazenda.gov.br/legislacao/ajustes/sinief/cfop_cvsn_70_vigente

Geramos uma tabela a partir do html existente em: https://www.lefisc.com.br/regulamentos/ricmsRR/anexoIX.asp



In [62]:
db.execute("""SELECT 
    cfop, 
    COUNT(*) AS occurrences
FROM mgi.itens
GROUP BY cfop
ORDER BY occurrences DESC
LIMIT 5""").df()

,cfop,occurrences
0,5102,4299469
1,6949,1807001
2,5405,1596958
3,6108,1124076
4,5949,1081100


In [49]:
db.execute("CREATE OR REPLACE SCHEMA raw")

In [50]:

db.execute("""create table raw.cfop as select * from read_xlsx('../data/inputs/rfb_cnpj/IT 2023.002-v1.00_Tabela_CFOP_Vigência_20230424.xlsx')""").df()

,Count
0,619


In [ ]:
db.query("SELECT * from raw.cfop limit 3").df()

,CFOP,Início de vigência,Fim de vigência,indNFe,indComunica,indTransp,indDevol,indRetor,indAnula,indRemes,indComb
0,1_101.00,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,1_102.00,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,1_111.00,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [90]:
db.execute("""
CREATE OR REPLACE TABLE raw.cfop2 AS         
SELECT
    CONCAT(cod_1, cod_2) AS cfop,
    * EXCLUDE (CFOP)
FROM (
    SELECT
        LEFT(CFOP::VARCHAR, 1) AS cod_1,
        LEFT(RIGHT(CFOP::VARCHAR, 5), 3) AS cod_2,
        *
    FROM raw.cfop
) t
ORDER BY cod_1, cod_2
         """)

In [99]:
db.query("FROM raw.cfop2").df()

,cfop,cod_1,cod_2,Início de vigência,Fim de vigência,indNFe,indComunica,indTransp,indDevol,indRetor,indAnula,indRemes,indComb
0,1101,1,101,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,1102,1,102,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,1111,1,111,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,1113,1,113,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,1116,1,116,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,7651,7,651,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00
615,7654,7,654,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00
616,7667,7,667,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00
617,7930,7,930,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [107]:
db.query("""
SELECT
  distinct cfop,
  COALESCE(indComunica, 0)::INT
+ COALESCE(indTransp,  0)::INT
+ COALESCE(indDevol,   0)::INT
+ COALESCE(indRetor,   0)::INT
+ COALESCE(indAnula,   0)::INT
+ COALESCE(indRemes,   0)::INT
+ COALESCE(indComb,    0)::INT  AS sum_ind
FROM raw.cfop2 where sum_ind > 1
ORDER BY sum_ind DESC, cfop """).df()

,cfop,sum_ind
0,1657,3
1,1660,3
2,1661,3
3,1662,3
4,2657,3
5,2660,3
6,2661,3
7,2662,3
8,5657,3
9,5660,3


In [78]:
db.query("select * from read_csv('../data/inputs/rfb_cnpj/cfop_anexoix.csv', header=true, sep=';', auto_type_candidates=['VARCHAR'])").df()

,CFOP,DESCRICAO,APLICACAO
0,1.000,ENTRADAS OU AQUISIÇÕES DE SERVIÇOS DO ESTADO,"Classificam-se, neste grupo, as operações ou p..."
1,1.100,"COMPRAS PARA INDUSTRIALIZAÇÃO, PRODUÇÃO RURAL,...",Classificam-se neste código as compras de merc...
2,1.101,COMPRAS PARA INDUSTRIALIZAÇÃO,Classificam-se neste código as compras de merc...
3,1.102,COMPRAS PARA COMERCIALIZAÇÃO,Classificam-se neste código as compras de merc...
4,1.111,COMPRAS PARA INDUSTRIALIZAÇÃO DE MERCADORIA RE...,Classificam-se neste código as compras efetiva...
...,...,...,...
737,7.651,Venda de combustível ou lubrificante de produç...,Classificam-se neste código as vendas de combu...
738,7.654,Venda de combustível ou lubrificante adquirido...,Classificam-se neste código as vendas de combu...
739,7.900,OUTRAS SAÍDAS DE MERCADORIAS OU PRESTAÇÕES DE ...,Classificam-se neste código as outras saídas d...
740,7.930,Lançamento efetuado a título de devolução de b...,Classificam-se neste código os lançamentos efe...


In [97]:
db.execute("""
CREATE OR REPLACE TABLE raw.cfop3 AS         
SELECT
    CONCAT(cod_1, cod_2) AS cfop,
    * EXCLUDE (CFOP)
FROM (
    SELECT
        LEFT(CFOP::VARCHAR, 1) AS cod_1,
        RIGHT(CFOP::VARCHAR, 3) AS cod_2,
        *
    FROM read_csv('../data/inputs/rfb_cnpj/cfop_anexoix.csv', header=true, sep=';', auto_type_candidates=['VARCHAR'])
) t
ORDER BY cod_1, cod_2
         """)

In [98]:
db.query("FROM raw.cfop3").df()

,cfop,cod_1,cod_2,DESCRICAO,APLICACAO
0,1000,1,000,ENTRADAS OU AQUISIÇÕES DE SERVIÇOS DO ESTADO,"Classificam-se, neste grupo, as operações ou p..."
1,1100,1,100,"COMPRAS PARA INDUSTRIALIZAÇÃO, PRODUÇÃO RURAL,...",Classificam-se neste código as compras de merc...
2,1101,1,101,COMPRAS PARA INDUSTRIALIZAÇÃO,Classificam-se neste código as compras de merc...
3,1102,1,102,COMPRAS PARA COMERCIALIZAÇÃO,Classificam-se neste código as compras de merc...
4,1111,1,111,COMPRAS PARA INDUSTRIALIZAÇÃO DE MERCADORIA RE...,Classificam-se neste código as compras efetiva...
...,...,...,...,...,...
737,7651,7,651,Venda de combustível ou lubrificante de produç...,Classificam-se neste código as vendas de combu...
738,7654,7,654,Venda de combustível ou lubrificante adquirido...,Classificam-se neste código as vendas de combu...
739,7900,7,900,OUTRAS SAÍDAS DE MERCADORIAS OU PRESTAÇÕES DE ...,Classificam-se neste código as outras saídas d...
740,7930,7,930,Lançamento efetuado a título de devolução de b...,Classificam-se neste código os lançamentos efe...


In [105]:
db.execute("SELECT distinct right(cfop::varchar,3) as cod from mgi.itens where cfop::varchar not in (select cfop from raw.cfop3) order by cod").df()

,cod
0,012
1,108
2,128
3,129
4,131
5,132
6,135
7,159
8,160
9,212


In [106]:
db.execute("SELECT distinct right(cfop::varchar,3) as cod from mgi.itens where cfop::varchar not in (select cfop from raw.cfop2) order by cod").df()

,cod
0,000
1,012
2,100
3,108
4,404
5,405
6,900
7,936
8,990


In [38]:
db.execute("SELECT DISTINCT CFOP FROM mgi.itens order by cfop").df()

,cfop
0,1101
1,1102
2,1111
3,1113
4,1116
...,...
401,6934
402,6936
403,6949
404,6990


In [36]:

db.query("""
SELECT 
*
FROM read_xlsx(
    -- '../data/inputs/rfb_cnpj/dicionario/160314_Tabela_CFOP.xlsx',
    '../data/inputs/rfb_cnpj/IT 2023.002-v1.00_Tabela_CFOP_Vigência_20230424.xlsx',
    header=true
)
WHERE CFOP = 1501
""").df()

,CFOP,Início de vigência,Fim de vigência,indNFe,indComunica,indTransp,indDevol,indRetor,indAnula,indRemes,indComb
0,1_501.00,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
db.execute("SELECT * FROM cfop where cfop").df()

In [43]:
db.execute("select chave_acesso, natureza_operacao, emitente, nome_emitente, destinatario, nome_destinatario from mgi.notas where emitente = destinatario").df()

,chave_acesso,natureza_operacao,emitente,nome_emitente,destinatario,nome_destinatario
0,31220310648539000296550010000040521418624296,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
1,31220310648539000296550010000040531305758359,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
2,53220300038174000496550010000122681001470415,"DOACAO, BONIFICACAO OU BRINDE",00038174000496,FUNDACAO UNIVERSIDADE DE BRASILIA,00038174000496,FUNDACAO UNIVERSIDADE DE BRASILIA
3,31220316888315000157550010000011411000011423,OUTRAS SAIDAS NAO ESPECIFICADAS,16888315000157,UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONH...,16888315000157,UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONH...
4,31220316888315000157550010000011421000011439,OUTRAS SAIDAS NAO ESPECIFICADAS,16888315000157,UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONH...,16888315000157,UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONH...
...,...,...,...,...,...,...
4446,31241116888315000157550010000021891000021906,OUTRAS SAIDAS NAO ESPECIFICADAS,16888315000157,UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONH...,16888315000157,UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONH...
4447,31241116888315000157550010000021901000021915,OUTRAS SAIDAS NAO ESPECIFICADAS,16888315000157,UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONH...,16888315000157,UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONH...
4448,33241133781055004980550000000674771023621231,Outra saída de mercadoria ou prestação de serviço,33781055004980,FARMANGUINHOS - FUNDAÇÃO OSWALDO CR,33781055004980,INSTITUTO DE TECNOLOGIA EM FARMACOS
4449,31241116888315000157550010000021911000021920,OUTRAS SAIDAS NAO ESPECIFICADAS,16888315000157,UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONH...,16888315000157,UNIVERSIDADE FEDERAL DOS VALES DO JEQUITINHONH...
